In [4]:
import pyspark.sql.functions as f
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.types as T
import pyspark
spark = SparkSession.builder \
    .master("local") \
    .appName("Time Series")